# Szimbólikus feladatmegoldás

Néha szükségünk van rá, hogy szimbólikusan számoljunk ki valamilyen feladatot. Ilyenkor sem kell feltétlen a Matlabért (pláne chatGPT-ért) szaladni, könnyedén ki tudunk számolni pythonban is meglehetősen összetett mérnöki feladatokat.

In [ ]:
# A szimbolikus feladatmegoldó csomag
from sympy import symbols, sin

x = symbols('x')
x * sin(x) + x**2


x**2 + x*sin(x)

Mivel a python szemében az x egy cimke amit program objektumokra ragasztgatunk és nem egy szimbólum, az első lépés, hogy létrehozunk egy `x` nevű szimbólumot és erre (praktikusan ugyanolyan nevű) cimkét ragasztunk.

A Symbol egy olyan objektum amire a műveletek másként működnek (épp úgy mint ahogy a `str` esetén a `+` összefűz és nem összead) jelen esetben "szimbólikusan".

Ha nem tetszik, hogy a szimbólikus változókat létre kell hozni a symbols függvénnyel, egyszerűen be is importálhatod őket az abc al-modulból:

In [ ]:
from sympy.abc import q, p
q * sin(x) + p * cos(x)

p*cos(x) + q*sin(x)

Nézzük mire képes a sympy a kalkulus terén (persze sokkal többre is képes):

In [ ]:
# számoljunk belőle deriváltat
from sympy import diff
diff(x * sin(x) + x**2)

x*cos(x) + 2*x + sin(x)

In [ ]:
# definiáljunk egy összetettebb saját függvényt
from sympy import diff, ln, integrate

def f(x):
  return x * sin(x) + x**2 / ln(x)

#és számoljunk ki a derivált és integrál függvényt is:
(diff(f(x)), integrate(f(x)))


(x*cos(x) + 2*x/log(x) - x/log(x)**2 + sin(x),
 -x*cos(x) + sin(x) + Ei(3*log(x)))

In [ ]:
# nem csak szimbólumokkal függvényekkel is dolgozhatunk!
from sympy import Function

f = Function('f')

diff( (f(x) - 3*x) / (2*f(x)) )

-(-3*x + f(x))*Derivative(f(x), x)/(2*f(x)**2) + (Derivative(f(x), x) - 3)/(2*f(x))

Egyszerűsíteni vagy egyenletek megoldani is tudunk

In [ ]:
# importáljunk még néhány sympy függvényt
from sympy import simplify, cos
x, y =  symbols('x y')
simplify( (x**2 + x)/(x*sin(y)**2 + x*cos(y)**2))

x + 1

In [ ]:
# És persze meg tudunk oldani egyenleteket ...
from sympy import solve

kif =  (x + y) * (x**2 - y)
solve(kif, x) # oldd meg a kifejezést x-re!

[-sqrt(y), sqrt(y), -y]

In [ ]:
# ha valamit kiszámoltunk csinálhatunk belőle hagyományos python függvényt.

szimbolikus = diff( x**2 / (sin(x) + x) )
szimbolikus

x**2*(-cos(x) - 1)/(x + sin(x))**2 + 2*x/(x + sin(x))

In [ ]:
from sympy import lambdify
# alakítsd hagyományos python függvénnyé a szimbólikus képletet:
python_függvény = lambdify(x, szimbolikus, "math")
python_függvény(0.23) # hagyományosan, float-okkal dolgozik
python_függvény(1)

0.6318578415774383

A sympy csak olyan egyszerűsítéseket (vagy egyenlőségeket) hajlandó értelmezni, amelyek mindig igazak. Sok azonosság van ami nem mindig igaz. Például matematikából ismerős lehet az azonosság, miszerint: $(x^a)^b = x^{ab}$. Csakhogy ez például x=-1, a=2, b=1/2 esetén nem igaz! Ha viszont $a$ és $b$ egész számok, akkor valóban igaz. Sympy lehetővé teszi, hogy feltételezéseket csatoljunk a szimbólumokhoz, például megszabhatjuk, hogy $a$ és $b$ csak egész lehet.

In [ ]:
a, b = symbols('a b')
simplify(2*(x**a)**b - x**(a*b))

-x**(a*b) + 2*(x**a)**b

In [ ]:
a, b = symbols('a b', integer=True)
simplify(2*(x**a)**b - x**(a*b))

x**(a*b)

Persze ez egy hatalmas csomag rengeteg lehetőséggel. Kombinatorika, Logika, mátrixok, számelmélet, fizikai számítások mind-mind a részét képezik. Ha bővebben érdekel itt tudsz utána nézni:
https://docs.sympy.org/latest/index.html

## Python Specialitások

A sympy egy python csomag, és mint ilyen nem tud (és nem is nagyon akar) változtatni a python alapvető elvein.  A pythonban a `=` jel, nem egyenlőséget takar hanem értékadást (vagy ugye még inkább felcimkézést), tehát olyasmit biztosan nem írhatunk, hogy:
```python
(x + 1)**2 = x**2 + 2*x + 1
```
Mert ez valami olyasmit akarna jelenteni, hogy `(x + 1)**2` cimkéhez akarjuk rendelni a jobb oldalt (de leginkább szintaktikai hiba).

Ugyanígy, a hatányozás jele `**` lesz, ahogy pythonban és nem `^` (ami a XOR operátor).

A dupla egyenlőség `==` már egyenlőséget jelent, viszont a sympy itt struktúrális egyenlőséget vizsgál, tehát akkor egyenlők, ha a szerkezetük is egyenlő az nem elég, hogy algebrailag egyenértékűek.

In [ ]:
(x + 1)**2 == x**2 + 2*x + 1

False

Ha "igazi" matematikai egyenlőségjelet szeretnénk, akkor az Eq konstruktort használjuk:

In [ ]:
from sympy import Eq
Eq(x**2 + 1, 10)

Eq(x**2 + 1, 10)

In [ ]:
solve(Eq(x**2 + 1, 10))

[-3, 3]

Ha csak az érdekel minket, hogy algebrailag egyenértékűek-e, a legtöbbször a legegyszerűbb csak simán kivonni a két kifejezést egymásból és megnézni, hogy nullát kapunk-e:

In [ ]:
kif1 =  (x + 1)**2
kif2 =  x**2 + 2*x + 1

# kif1 == kif2 ?
simplify(kif1-kif2)

0

In [ ]:
# vagy megnézhetjük az equals metódussal is:
kif1.equals(kif2)

True

A háttérben a Sympy mindent átkonvertál "sympy objektummá" és a rajtuk végzett műveletek eredménye is az lesz.

In [ ]:
eredmény = simplify(x - x + 1 )
eredmény

1

In [ ]:

type(eredmény)

sympy.core.numbers.One

In [ ]:
# osztásnál érdemes odafigyelni, mert a python `1/3` egy lebegőpontos szám
# (ami nem pontos) míg sympy-ben az Integer(1) / Integer(3) egy nagyon is
# egzakt arány.

x + 1/3  # (az 1/3-ot előbb értékeli ki a python és az egy 'pontatlan' float)

x + 0.333333333333333

In [ ]:
# ezzel szemben:
from sympy import Integer
x + Integer(1) / Integer(3)

x + 1/3

In [ ]:
# vagy simán racionális számokat használunk:
from sympy import Rational
x + Rational(1,3)

x + 1/3